In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
import xgboost as xgb
import joblib
import pickle
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
import statsmodels.api as sma
import statsmodels.stats.outliers_influence  as smo
from scipy.stats import uniform, randint
from model_mngr import *
import model_mngr
from split_merge import *
import split_merge
from add_indicators import *
import add_indicators
from maria_import_export import *
from importlib import reload
import pandas as pd
import numpy as np
import random as rd
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
from boruta import BorutaPy
import maria_import_export
reload(maria_import_export)
reload(add_indicators)
reload(split_merge)
reload(model_mngr)


Load data for CW8 ETF from Maria
Add indicators and labels and save the dataset on disk

In [ ]:
PATH_DATA = "../../../Data/DTS_FULL/"
PATH_MODEL = "../../../Data/Models/"
symb = "CW8"
dts_name = "DCA_CLOSE_1D_V1"
rnd_key = int(get_conf("RANDOM_KEY"))
df = pd.DataFrame()
if "con" in locals():
    close_connection(con)
con = get_connection()
df = get_candles_to_df(con=con, symbol=symb, only_close=True)
df = add_indicators_to_df(con=con, df_in=df, dts_name=dts_name)
df.sort_index(inplace=True)
#df.round(5).to_csv(PATH_DATA+dts_name+"_full.zip", sep=",")


Clean the dataset droping useless features
Split the dataset by labels, train, val, conf part

In [ ]:
df = drop_indicators_by_type(
    con=con, df_in=df, dts_name=dts_name, symbol=symb, ind_type=0)
list_label = get_ind_list_by_type_for_dts(
    con=con, dts_name=dts_name, symbol=symb, ind_type=2)
dict_split = split_df_by_label_strat(
    df_in=df, list_label=list_label['LABEL'].tolist(), split_timeframe="Q")
print(list_label)
lab = list_label['LABEL'][3]
df_test = dict_split['df_'+lab+'_train']
df_test.sort_index(inplace=True)
print(df_test.loc[:, lab].dropna().iloc[[0, -1]])


selection of the df studied and plot data to check

In [ ]:
# dict_split.keys()
# lab_studied="lab_perf_21d"
lab_studied = "lab_perf_21d"
df_studied = "df_"+lab_studied
df_selected = dict_split[df_studied+'_train']
df_valid = dict_split[df_studied+'_valid']
df_confirm = dict_split[df_studied+'_confirm']
list_feat = get_ind_list_by_type_for_dts(
    con=con, dts_name=dts_name, symbol=symb, ind_type=1)
list_feat = list_feat['LABEL'].tolist()


Check data for each  label

In [ ]:
for lab in list_label["LABEL"].to_list():
    print(f"label : "+lab)
    print(df.loc[:, lab].dropna().iloc[[0, -1]])
(df.loc[:, list_label["LABEL"]]).hist(figsize=(8, 10))
# df_selected.hist(figsize=(15,20))
df.head()


split dataframe into X,y

In [ ]:
df_x_train, col_y_train = split_df_x_y(
    df_in=df_selected, list_features=list_feat, str_label=lab_studied, drop_na=True)
df_x_valid, col_y_valid = split_df_x_y(
    df_in=df_valid, list_features=list_feat, str_label=lab_studied, drop_na=True)


Analysis of train set, heatmap correlation 

In [ ]:
corr_train = df_x_train.corr()
fig, ax = plt.subplots(figsize=(6, 6))
corr_train.replace(1,0,inplace=True)
corr_train=corr_train.applymap(lambda x : None if x< 0.9 and x>-0.9 else x)
corr_train.dropna(axis=0,how='all',inplace=True)
corr_train.dropna(axis=1,how='all',inplace=True)
sns.heatmap(corr_train, annot=False, cmap='coolwarm', vmin=-1, vmax=1, ax=ax)


Variance Inflation Factor

In [ ]:
df_x_train_VIF = sma.add_constant(df_x_train)
vif = pd.DataFrame()
vif["feature"]=df_x_train_VIF.columns
vif["VIF"]= [smo.variance_inflation_factor(df_x_train_VIF.values, i) for i in range(df_x_train_VIF.shape[1])]
print(vif)


Feature analysis with Boruta

In [ ]:
rf_bo = RandomForestRegressor(n_estimators=100, n_jobs=-1)
boruta_selector = BorutaPy(rf_bo, n_estimators='auto',
                           verbose=1, random_state=int(rnd_key))
boruta_selector.fit(df_x_train.values, col_y_train.values)
selected_features = df_x_train.columns[boruta_selector.support_]
print(selected_features)


Hyperparameter tuning for Random Forest with random search CV

In [ ]:
# number of trees in the foreset
n_estimators = [int(x) for x in np.linspace(start=50, stop=200, num=12)]
# max number of features considered for splitting a node
max_features = ['sqrt']
# max number of levels in each decision tree
max_depth = [int(x) for x in np.linspace(2, 6, num=5)]
# min number of data points placed in a node before the node is split
min_samples_split = np.linspace(0.1, 1.0, 10, endpoint=True)
# min number of data points allowed in a leaf node
min_samples_leaf = np.linspace(0.1, 0.5, 5, endpoint=True)

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

rf = RandomForestRegressor()

rf_fitted = randomizedsearch_cv_fit_report(estimator=rf, param_distributions=random_grid,
                                           x_train=df_x_train, y_train=col_y_train, random_state=int(rnd_key), n_iter=200, n_top=5)

# 21 : n_estimators': 90, 'min_samples_split': 0.1, 'min_samples_leaf': 0.1, 'max_features': 'sqrt', 'max_depth': 6
# 62 : 'n_estimators': 172, 'min_samples_split': 0.5, 'min_samples_leaf': 0.2, 'max_features': 'sqrt', 'max_depth': 5


check parameters and test with validation dataset and show feature importance

In [ ]:
rand_frst_reg = RandomForestRegressor(n_estimators=172, bootstrap=True, criterion="squared_error",
                                      random_state=rnd_key, max_depth=5, min_samples_split=0.5, min_samples_leaf=0.2, max_features="sqrt")
rand_frst_reg.fit(df_x_train, col_y_train)


print(f"Training score : {rand_frst_reg.score(df_x_train, col_y_train)}")

y_pred_valid = rand_frst_reg.predict(df_x_valid)
print(
    f"Random Forest regressor score:{rand_frst_reg.score(df_x_valid,col_y_valid)}")

feat_imp = rand_frst_reg.feature_importances_
std_feat_imp = np.std(
    [tree.feature_importances_ for tree in rand_frst_reg.estimators_], axis=0)
pd.Series(feat_imp, index=df_x_train.columns).sort_values(ascending=False)


save model

In [ ]:
ext_pkl = ".pkl"
rf_suffix = "_rf_reg_v1"
joblib.dump(rand_frst_reg, PATH_MODEL+symb+"_"+lab_studied+rf_suffix+ext_pkl)


Same thing with XGBoost
Hyperparameter tuning for XGBoost with random search CV

In [ ]:

# 5 min 30 s !!

random_grid = {
    # default 1 subsample ratio of columns when constructing each tree
    "colsample_bytree": [x for x in np.linspace(0.5, 1, num=6)],
    # default 0 minimum loss reduction to make a further patition on a leaf
    "gamma": [x for x in np.linspace(0, 0.1, num=3)],
    "learning_rate": [x for x in np.linspace(0.05, 0.2, num=8)],  # default 0.1
    # default 6 max depth of the tree
    "max_depth": [int(x) for x in np.linspace(2, 8, num=5)],
    # default 100
    "n_estimators": [int(x) for x in np.linspace(start=50, stop=200, num=7)],
    # default 1 subsample ratio of the training instances
    "subsample": [x for x in np.linspace(0.4, 1, num=6)],
    # default 1 minimum sum of instance weight needed in a child
    "min_child_weight": [x for x in np.linspace(0.8, 2, num=7)]
}

xgb_reg = xgb.XGBRegressor()

xgb_fitted = randomizedsearch_cv_fit_report(estimator=xgb_reg, param_distributions=random_grid,
                                            x_train=df_x_train, y_train=col_y_train, random_state=int(rnd_key), n_iter=200, n_top=5)

# 21 Parameters: {'subsample': 1.0, 'n_estimators': 125, 'min_child_weight': 1.0, 'max_depth': 2, 'learning_rate': 0.1142857142857143, 'gamma': 0.1, 'colsample_bytree': 0.7}
# 21 {'subsample': 1.0, 'n_estimators': 75, 'min_child_weight': 1.4, 'max_depth': 6, 'learning_rate': 0.15714285714285717, 'gamma': 0.1, 'colsample_bytree': 0.6}
# 62 'subsample': 1.0, 'n_estimators': 125, 'min_child_weight': 1.4, 'max_depth': 3, 'learning_rate': 0.05, 'gamma': 0.05, 'colsample_bytree': 1.0


In [ ]:
# avoir un gamma <=0.3 voire 0.1 et meme 0

params = {
    "learning_rate": 0.05,
    "subsample": 1.0,
    "gamma": 0.05,
    "colsample_bytree": 1.0,
    "max_depth": 3,
    "n_estimators": 125,
    "min_child_weight": 1.4
}

# params = {
#     "learning_rate": 0.11,
#     "subsample": 1.0,
#     "gamma": 0.1,
#     "colsample_bytree":0.7,
#     "max_depth":2,
#     "n_estimators":100,
#     "min_child_weight":1.0
# }
xgb_reg = xgb.XGBRegressor(random_state=int(rnd_key),
                           learning_rate=params['learning_rate'], subsample=params['subsample'], gamma=params['gamma'],
                           colsample_bytree=params['colsample_bytree'], max_depth=params['max_depth'],
                           n_estimators=params['n_estimators'], min_child_weight=params['min_child_weight'], verbosity=1)

# xgb_reg=xgb.XGBRegressor()
xgb_reg.fit(df_x_train, col_y_train)

print(f"Training score : {xgb_reg.score(df_x_train, col_y_train)}")

#y_pred_valid = xgb_reg.predict(df_x_valid)

print(
    f"Accuracy XGBoost regressor Validation:{xgb_reg.score(df_x_valid,col_y_valid)}")

# print(xgb_reg.feature_importances_)
xgb.plot_importance(xgb_reg)


In [ ]:
ext_pkl = ".pkl"
xgb_suffix = "_xgb_reg_v1"
joblib.dump(xgb_reg, PATH_MODEL+symb+"_"+lab_studied+xgb_suffix+ext_pkl)


Same thing with Ridge regression 
Hyperparameter tuning for Logistic Regression with random search CV

In [ ]:
random_grid = {
    "solver": ["svd", "cholesky", "lsqr", "sag"],
    "alpha": [x for x in np.logspace(1e-5, 100, 20)],
    "fit_intercept": [True, False],
    "normalize": [True, False]
}
lin_reg = Ridge()

lin_fitted = randomizedsearch_cv_fit_report(estimator=lin_reg, param_distributions=random_grid,
                                            x_train=df_x_train, y_train=col_y_train, random_state=int(rnd_key), n_iter=200, n_top=5)


In [ ]:
params = {
    "solver": "cholesky",
    "normalize": False,
    "fit_intercept": True,
    "alpha": 183302
}

lin_reg = Ridge(random_state=int(rnd_key),
                solver=params['solver'], normalize=params['normalize'], fit_intercept=params['fit_intercept'],
                alpha=params['alpha'])

lin_reg.fit(df_x_train, col_y_train)

print(f"Training score : {lin_reg.score(df_x_train, col_y_train)}")

print(
    f"Accuracy Ridge regressor Validation:{lin_reg.score(df_x_valid,col_y_valid)}")

print(lin_reg.coef_)


plt.bar(df_x_train.columns, lin_reg.coef_)
plt.xticks(range(len(df_x_train.columns)), df_x_train.columns, rotation=90)
plt.ylabel('Coefficient')
plt.title('Ridge features importance')
plt.show()
